In [2]:
import pandas as pd
import matplotlib.pyplot as plt
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_train.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [3]:
x = df_train.drop('ACTION',axis=1)      #input features
y = df_train['ACTION']  


In [4]:
test_input = df_test.drop('id',axis=1)      #test input containing features
test_input.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,78766,72734,118079,118080,117878,117879,118177,19721,117880
1,40644,4378,117961,118327,118507,118863,122008,118398,118865
2,75443,2395,117961,118300,119488,118172,301534,249618,118175
3,43219,19986,117961,118225,118403,120773,136187,118960,120774
4,42093,50015,117961,118343,119598,118422,300136,118424,118425


In [5]:
from sklearn.model_selection import train_test_split
#splitting in 20:80 for validation of data
x_train, x_valid, y_train, y_valid = train_test_split(x,y,test_size=0.2,random_state=1)

In [6]:
from catboost import CatBoostClassifier                 #using catboost because its fast and doesnt require pre-processing
from sklearn.metrics import f1_score

In [7]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()

In [8]:
x_train,y_train = oversample.fit_resample(x_train,y_train)

In [9]:
from catboost import CatBoostClassifier                 #using catboost because its fast and doesnt require pre-processing
from sklearn.metrics import f1_score

In [10]:
cat_features = list(range(x_train.columns.size))        #classified features on which our model will train

model = CatBoostClassifier(
    loss_function='CrossEntropy',
    eval_metric="F1",
    random_seed=1,
    cat_features = cat_features
)

model.fit(x_train,y_train,eval_set=(x_valid,y_valid),use_best_model=True)

0:	learn: 0.8874226	test: 0.9559120	best: 0.9559120 (0)	total: 248ms	remaining: 4m 7s
1:	learn: 0.8897277	test: 0.9633951	best: 0.9633951 (1)	total: 382ms	remaining: 3m 10s
2:	learn: 0.8896046	test: 0.9644711	best: 0.9644711 (2)	total: 459ms	remaining: 2m 32s
3:	learn: 0.8878489	test: 0.9649725	best: 0.9649725 (3)	total: 546ms	remaining: 2m 15s
4:	learn: 0.8890593	test: 0.9628978	best: 0.9649725 (3)	total: 648ms	remaining: 2m 8s
5:	learn: 0.8886670	test: 0.9619848	best: 0.9649725 (3)	total: 734ms	remaining: 2m 1s
6:	learn: 0.8886841	test: 0.9619848	best: 0.9649725 (3)	total: 821ms	remaining: 1m 56s
7:	learn: 0.8881964	test: 0.9565988	best: 0.9649725 (3)	total: 911ms	remaining: 1m 52s
8:	learn: 0.8882007	test: 0.9565147	best: 0.9649725 (3)	total: 1.02s	remaining: 1m 52s
9:	learn: 0.8890300	test: 0.9583668	best: 0.9649725 (3)	total: 1.11s	remaining: 1m 50s
10:	learn: 0.8931844	test: 0.9611285	best: 0.9649725 (3)	total: 1.26s	remaining: 1m 53s
11:	learn: 0.8975712	test: 0.9626363	best: 0.

KeyboardInterrupt: 

In [11]:
predictions = model.predict(test_input)

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.

In [ ]:
result = pd.DataFrame({'Id':df_test['id'],'Action':predictions})
result.to_csv('Submission10.csv',index=False)
#BY CATBOOST

In [ ]:
#BY ADABOOST
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(learning_rate=10**-3,n_estimators=100,random_state=1)
clf.fit(x_train,y_train)

AdaBoostClassifier(learning_rate=0.001, n_estimators=100, random_state=1)

In [ ]:
clf.predict(test_input)

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [ ]:
clf.score(x_valid,y_valid)

0.7509917607567898

In [ ]:
result = pd.DataFrame({'Id':df_test['id'],'Action':clf.predict(test_input)})
result.to_csv('Submission11.csv',index=False)

In [ ]:
!pip install xgboost


In [19]:
import xgboost as xgb
model_xgb = xgb.XGBClassifier(learning_rate=1,n_estimators=2000,eval_metric='auc')
eval_set = [(x_valid,y_valid)]
model_xgb.fit(x_train,y_train,eval_set=eval_set)

[0]	validation_0-auc:0.66212
[1]	validation_0-auc:0.68397
[2]	validation_0-auc:0.68958
[3]	validation_0-auc:0.71112
[4]	validation_0-auc:0.72827
[5]	validation_0-auc:0.75032
[6]	validation_0-auc:0.74814
[7]	validation_0-auc:0.74637
[8]	validation_0-auc:0.75741
[9]	validation_0-auc:0.77263
[10]	validation_0-auc:0.77462
[11]	validation_0-auc:0.78573
[12]	validation_0-auc:0.78936
[13]	validation_0-auc:0.79128
[14]	validation_0-auc:0.79754
[15]	validation_0-auc:0.80458
[16]	validation_0-auc:0.80268
[17]	validation_0-auc:0.80383
[18]	validation_0-auc:0.80398
[19]	validation_0-auc:0.80371
[20]	validation_0-auc:0.80277
[21]	validation_0-auc:0.80022
[22]	validation_0-auc:0.79993
[23]	validation_0-auc:0.79935
[24]	validation_0-auc:0.80102
[25]	validation_0-auc:0.80214
[26]	validation_0-auc:0.80278
[27]	validation_0-auc:0.80484
[28]	validation_0-auc:0.80256
[29]	validation_0-auc:0.80219
[30]	validation_0-auc:0.80305
[31]	validation_0-auc:0.80272
[32]	validation_0-auc:0.80548
[33]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=1, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=2000, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [20]:
result = pd.DataFrame({'Id':df_test['id'],'Action':model_xgb.predict(test_input)})
result.to_csv('Submission12.csv',index=False)

In [22]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

sc.fit(x_train)

X_train_std = sc.transform(x_train)

sc.fit(test_input)

X_train_std = sc.transform(test_input)

sc.fit(x_valid)

X_train_std = sc.transform(x_valid)


In [23]:
model_xgb = xgb.XGBClassifier(learning_rate=1,n_estimators=2000,eval_metric='auc')
eval_set = [(x_valid,y_valid)]
model_xgb.fit(x_train,y_train,eval_set=eval_set)

[0]	validation_0-auc:0.66212
[1]	validation_0-auc:0.68397
[2]	validation_0-auc:0.68958
[3]	validation_0-auc:0.71112
[4]	validation_0-auc:0.72827


C:\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[5]	validation_0-auc:0.75032
[6]	validation_0-auc:0.74814
[7]	validation_0-auc:0.74637
[8]	validation_0-auc:0.75741
[9]	validation_0-auc:0.77263
[10]	validation_0-auc:0.77462
[11]	validation_0-auc:0.78573
[12]	validation_0-auc:0.78936
[13]	validation_0-auc:0.79128
[14]	validation_0-auc:0.79754
[15]	validation_0-auc:0.80458
[16]	validation_0-auc:0.80268
[17]	validation_0-auc:0.80383
[18]	validation_0-auc:0.80398
[19]	validation_0-auc:0.80371
[20]	validation_0-auc:0.80277
[21]	validation_0-auc:0.80022
[22]	validation_0-auc:0.79993
[23]	validation_0-auc:0.79935
[24]	validation_0-auc:0.80102
[25]	validation_0-auc:0.80214
[26]	validation_0-auc:0.80278
[27]	validation_0-auc:0.80484
[28]	validation_0-auc:0.80256
[29]	validation_0-auc:0.80219
[30]	validation_0-auc:0.80305
[31]	validation_0-auc:0.80272
[32]	validation_0-auc:0.80548
[33]	validation_0-auc:0.80626
[34]	validation_0-auc:0.81145
[35]	validation_0-auc:0.81317
[36]	validation_0-auc:0.81396
[37]	validation_0-auc:0.81471
[38]	validation

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=1, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=2000, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [24]:
result = pd.DataFrame({'Id':df_test['id'],'Action':model_xgb.predict(test_input)})
result.to_csv('Submission13.csv',index=False)